### I thought after discussing it in Office Hours that it would work, but unfortunately it still doesn't work. There is an error because there is no Coronavirus Task Force briefing on the index page. It is only on subsequent pages. I tried to fix it but I couldn't figure it out. I am using data from the end of April, so it doesn't affect my blog post, but this code does not run.

# Collecting WH briefing transcripts

* Want healthcare tag briefings since Jan 1, 2020 from 
    * https://www.whitehouse.gov/briefings-statements/?issue_filter=healthcare
    * These include both _Statements_ and _Remarks_
        * Is it just _Remarks_ and/or those that say "Press Briefing" in title, e.g.
          > Remarks by President Trump, Vice President Pence, and Members of the Coronavirus Task Force in Press Briefing | April 17, 2020
          >
          > Remarks by President Trump, Vice President Pence, and Members of the Coronavirus Task Force in Press Briefing
          >
          > Remarks by President Trump in Press Briefing
          
          etc..
          


### Examining the structure of the HTML on briefing index pages


* On the index pages each transcript is described and linked in an `<article>` HTML element with class attributes: `briefing-statement`.


* An `<h2>` element with a `briefing-statement__title` contains both:
    1. the link to transcript
    2. the title that can be tested to see if it contains key terms, e.g. _Coronavirus_ or _Press Briefing_.


* Also the `<time>` element gives a date for the briefing

    ```
    <article class="briefing-statement briefing-statement--results">
        <div class="briefing-statement__content">
                        <p class="briefing-statement__type">
                    Remarks			</p>

            <h2 class="briefing-statement__title"><a href="https://www.whitehouse.gov/briefings-statements/remarks-president-trump-vice-president-pence-members-coronavirus-task-force-press-briefing-april-17-2020/">Remarks by President Trump, Vice President Pence, and Members of the Coronavirus Task Force in Press Briefing | April 17, 2020</a></h2>

            <div class="meta meta--left">
                                <div class="meta__issue">
                        <p class="issue-flag issue-flag--left">
                <a href="https://www.whitehouse.gov/issues/healthcare/">
                Healthcare		</a>
            </p>
                    </div>
                            <p class="meta__date">
                    <time>Apr 17, 2020</time>
                </p>
            </div>
        </div>
    </article>
    ```

* Then to move to the next page of briefing descriptions and links we need to location the __Next__ link:


```
<a class="pagination__next" 
   href="https://www.whitehouse.gov/briefings-statements/page/2/?issue_filter=healthcare">
   Next
</a>
```

### Setup

In [22]:
import requests
from bs4 import BeautifulSoup
import time
from datetime import datetime
import json

### Example of processing the index page

1. Download the HTML of the page using `requests.get` function
2. Create a `BeautifulSoup` object from this `str` object
3. Find all `<article class="briefing-statement">` elements
4. Filter these elements based on the title for the string `Coronavirus Task Force`

In [23]:
start_url = "https://www.whitehouse.gov/briefings-statements?issue_filter=healthcare"

1. Download HTML

In [24]:
resp = requests.get(start_url)
resp

<Response [200]>

2. Create `BeautifulSoup` object from the string contents

In [25]:
doc = BeautifulSoup(resp.text, 'html.parser')

3. Find `<article class="briefing-state">` elements

In [26]:
briefings = doc.find_all('article', attrs={'class': 'briefing-statement'})

In [6]:
len(briefings)

10

* Ten briefings on this page


* Loop over briefings and look at the title and the time

In [7]:
for briefing in briefings:
    print(briefing.find('h2').text)
    time = briefing.find('time')
    print(time.text, '\n')

Presidential Message on National Women’s Health Week, 2020
May 10, 2020 

Remarks by Vice President Pence in Discussion with Faith Leaders | Urbandale, IA
May 8, 2020 

Remarks by President Trump in Meeting with Republican Members of Congress
May 8, 2020 

President Donald J. Trump Approves Seminole Tribe of Florida Disaster Declaration
May 8, 2020 

Remarks by President Trump in Meeting with Governor Abbott of Texas
May 7, 2020 

Remarks by President Trump and Vice President Pence at a Meeting with Governor Reynolds of Iowa
May 6, 2020 

Remarks by President Trump at Signing of a Proclamation in Honor of National Nurses Day
May 6, 2020 

Remarks by President Trump in Roundtable Discussion on Supporting Native Americans | Phoenix, AZ
May 5, 2020 

Remarks by President Trump at Honeywell International Inc. Mask Production Facility | Phoenix, AZ
May 5, 2020 

President Donald J. Trump is Protecting the Native American Community as We Combat the Coronavirus
May 5, 2020 



* Create a list of dictionaries, where each dictionary has:
   * `title`
   * `date`
   * `url`  

In [8]:
briefing_links = []

for briefing in briefings:
    title = briefing.find('h2').text
    if title.count('Coronavirus Task Force'):
        date = briefing.find('time').text
        url = briefing.find('h2').find('a').attrs['href']
        
        bdata = {'title': title,
                 'date': date,
                 'url': url
                }
        
        briefing_links.append(bdata)

In [9]:
len(briefing_links)

0

In [10]:
briefing_links

[]

### Get briefing transcript

* Looking at a sample transcript page:
    * https://www.whitehouse.gov/briefings-statements/remarks-president-trump-vice-president-pence-members-coronavirus-task-force-press-briefing-27/
    
    
* The transcript is inside the:

    ```
    <div class="page-content__content editor">
    ```
		
* In a series of `<p>` elements

In [21]:
r=requests.get(briefing_links[0]['url'])

IndexError: list index out of range

In [14]:
tdoc=BeautifulSoup(r.text, 'html.parser')

NameError: name 'r' is not defined

In [15]:
paras=tdoc.find('div', attrs={"class": "page-content__content"}).find_all('p')

NameError: name 'tdoc' is not defined

In [16]:
print('\n\n'.join([p.text for p in paras]))

NameError: name 'paras' is not defined

* Add transcript into the `briefing_links`


In [17]:
for briefing in briefing_links:
    url = briefing['url']
    print('Getting transcript text from', url)
    r = requests.get(url)
    d = BeautifulSoup(r.text, 'html.parser')
    
    paras=d.find('div', attrs={"class": "page-content__content"}).find_all('p')
    
    transcript_text = '\n\n'.join([p.text for p in paras])
    briefing['text'] = transcript_text
    

In [18]:
briefing_links

[]

### Putting it all together



In [14]:
def get_links(link_url, date_stop=None):
    
    resp = requests.get(link_url)
    
    while not resp.ok:
        time.sleep(1)
        resp = requests.get(link_url)

    
    doc = BeautifulSoup(resp.text, 'html.parser')
    
    briefings = doc.find_all('article', attrs={'class': 'briefing-statement'})        
    
    briefing_links = []

    for briefing in briefings:
        title = briefing.find('h2').text
        date = briefing.find('time').text
        dt = datetime.strptime(date,'%b %d, %Y')


        if title.count('Coronavirus Task Force') and (date_stop and dt>=date_stop):
                
            url = briefing.find('h2').find('a').attrs['href']

            bdata = {'title': title,
                     'date': date,
                     'url': url
                    }

            briefing_links.append(bdata)
            
    # check for next link else return None
    next_link = None if (date_stop and dt<=date_stop) else doc.find('a', attrs={'class': 'pagination__next'})
    
    next_page = None if not next_link else next_link.attrs['href']
    
    
    return briefing_links, next_page

In [15]:
links, next_url = get_links(start_url, date_stop=datetime.strptime('Jan 1, 2020','%b %d, %Y'))

In [16]:
next_url

'https://www.whitehouse.gov/briefings-statements/page/2/?issue_filter=healthcare'

In [17]:
briefing_links=[]

next_url = start_url
#https://commjhub.asc.upenn.edu/user/mathaim/notebooks/comm313_S20/comm313_S20_Assignment8/collecting_transcripts_of_WH_briefings.ipynb
while next_url:
    print(next_url)
    links, next_url = get_links(next_url, date_stop=datetime.strptime('Jan 1, 2020','%b %d, %Y'))
    print(links)
    briefing_links.extend(links)
    time.sleep(1)

https://www.whitehouse.gov/briefings-statements?issue_filter=healthcare
[]


TypeError: 'NoneType' object is not callable

In [186]:
len(briefing_links)

53

* There are 51 briefings containing the string _Coronavirus Task Force_ on or after Jan 1, 2020


* Now add the transcripts for each

In [18]:
for blink in briefing_links:
    url = blink['url']
    print('Getting transcript from', url)
    r = requests.get(url)
    while not r.ok:
        time.sleep(1)
        r = requests.get(url)

    tdoc=BeautifulSoup(r.text, 'html.parser')
    paras=tdoc.find('div', attrs={"class": "page-content__content"}).find_all('p')
    
    text = '\n\n'.join([p.text for p in paras])
    
    blink['text'] = text
    time.sleep(1)

* Save the transcripts to a JSON file

In [19]:
with open('data/briefing_transcripts.json','w') as out:
    out.write(json.dumps(briefing_links, indent=4))

In [20]:
briefing_links[0]['date'].split(' ')[0]

IndexError: list index out of range

In [220]:
apr_briefings = [item for item in briefing_links
                    if item['date'].split()[0]=='Apr'
                ]

In [221]:
len(apr_briefings)

23

In [222]:
apr_briefings

[{'date': 'Apr 27, 2020',
  'text': 'Rose Garden\n\n5:38 P.M. EDT\n\nTHE PRESIDENT:\xa0 Thank you very much.\xa0 Thank you.\xa0 Today I’d like to provide you with an update in our war against the coronavirus.\xa0 Thanks to our comprehensive strategy and extraordinary devotion to our citizens — we’ve had such tremendous support all over — we continue to see encouraging signs of progress.\n\nCases in New York area, New Orleans, Detroit, Boston, and Houston are declining.\xa0 Denver, Seattle, Philadelphia, Washington, D.C., Atlanta, Nashville, Indianapolis, and St. Louis are all stable and declining.\xa0 All parts of the country are either in good shape, getting better.\xa0 In all cases, getting better.\xa0 And we’re seeing very little that we’re going to look at as a superseding hotspot.\xa0 Things are moving along.\xa0 Really, a horrible situation that we’ve been confronted with, but they’re moving along.\n\nAs we express our gratitude for these hard-fought gains, however, we continue t